In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/TalkLikeMom/src/Classifier

/content/drive/MyDrive/PHD/TalkLikeMom/src/Classifier


In [ ]:
!pip install -q transformers pythainlp datasets evaluate sentencepiece
# !pip install -q accelerate -U
!pip install -q transformers[torch]
!pip install -q shap nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.3 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append('.')

In [ ]:
from data_loader import get_task1_conver, get_task2_conver
from utils import dump_jsonl, load_jsonl


import pickle

def load_shap_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

def save_shap_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)

In [ ]:
import pandas as pd

In [ ]:
from transformers import AutoTokenizer
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens": ["usr", "sys", "rep"]})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

In [ ]:
# "DONE"

## Load Lexicons

In [ ]:
from pythainlp.tokenize import word_tokenize
import numpy as np

In [ ]:
import json
with open("../words.json", encoding="utf-8") as fin:
    raw = json.load(fin)
    thaidict_royal = set()
    for k in raw:
        thaidict_royal.update(raw[k])

In [ ]:
lexicons_arr = load_jsonl("../lexicons.jsonl")

Loaded 25603 records from ../lexicons.jsonl


In [ ]:
from collections import defaultdict
tags = set()
lexicons = {}
lexicons_keys = defaultdict(list)

for key, values  in lexicons_arr:
    if len(key) <= 1:
        continue

    key = key.lower()
    if key.endswith("rep"):
        key = key.replace("rep", "")

    w = word_tokenize(key)

    lexicons_keys[w[0]].append(key)

    tag = [t for t in values["tags"] if not t.startswith("cat:")]
    lexicons[key] = tag
    tags.update(tag)

In [ ]:
# cc = 0
# for k in lexicons:
#     for t in lexicons[k]:
#         if t=="transliterated":
#             cc += 1
# cc

## Calculate Shapley

In [ ]:
metric_names = {
    "Reference" : {
#         "all": "All words",
        "pertoken": "Average per token"
    },
#     "Linguistic Complexity" : {
#         "nunique": "Vocabulary size",
#         "nthai": "Thai words",
#         "nnotthai": "Non-Thai words",
#         "nlongword": "Long words",
#         "ndict": "Dictionary words",
#         "transliterated": "Transliteration",
#     },
    "Pronoun": {
        "pronoun": "All pronoun",
        "pronoun_1st": ">> 1st person pronoun",
        "pronoun_2nd": ">> 2nd person pronoun",
        "pronoun_3rd": ">> 3rd person pronoun",
        "pronoun_singular": ">> Singular pronoun",
        "pronoun_plural": ">> Plural pronoun",
        "pronoun_misspelling": ">> Pronoun in non-standard spelling",
    },

    "Sentence-ending Particles": {
        "particles": "All particles",
        "particles_SARP": ">> Socially-related particles",
        "particles_notSARP": ">> Non-socially-related particles",
        "particles_misspelling": ">> Particle in non-standard spelling",
    },

#     "Sentiment-related": {
#         "sentiment": "Sentiment words",
#         "sentiment_positive": ">> Positive words",
#         "sentiment_negative": ">> Negative words",
#     },

    "Spelling Variation": {
        "misspelling": "All spelling variation",
        "misspelling_common": ">> Common misspelt words",
        "misspelling_intention": ">> Morphophonemic variation",
        "misspelling_shorten": ">> Simplified variation",
        "nrepeat": ">> Repeated characters",
#         "nemoji": ">> Emoji",
#         "abbr": "Abbreviation",
#         "slang": "Slang",
#         "swear": "Swear words"
    }
}

In [ ]:
def map_token_2_words(words, shap_tokens, debug=False):
    tokens = [w for w, _ in  shap_tokens]
    values = np.array([v for _, v in  shap_tokens])

    idxs = []
    sidx = 0
    windows = 10

    newwords = []
    newtokens = []
    newvalues = []


    widx = 0
    w = ""
    while widx < len(words):
        w += words[widx]
        if sidx >= len(tokens):
#             print(newwords)
#             assert(False)
            break

#         print(widx, w, sidx, tokens[sidx])
#         break
        s = ""
        matched = False
        for tidx in range(sidx, min(sidx+windows, len(tokens))):
            s += tokens[tidx]
            if s==w:
                matched = True
                break

        if matched:
            if debug:
                print("MATCHED", w)
            idxs.append([sidx, tidx+1])
            newwords.append(w)
            newtokens.append("".join(tokens[sidx:tidx+1]))
            newvalues.append(values[sidx:tidx+1].sum())
            sidx = tidx+1
            w = ""
            widx += 1
            continue

        if debug:
            print("NOT MATCHED", w, s)

        if not s.startswith(w):
            sidx += 1
            w = ""
#             print("SKIP TOKEN")
            continue
        else:
            widx += 1
#             print("MERGE WORDS")
            continue

    if debug:
        print(newwords)
        print(newtokens)


    return newtokens, newvalues


In [ ]:
# def outputs_to_dict(outputs):
#     coefs = {}
#     for s in outputs[0]:
#         for feat in outputs[0][s]:
#             if feat not in coefs:
#                 coefs[feat] = (0, 0)

#             val1, n1 = outputs[0][s][feat]
#             val2, n2 = coefs[feat]

#             if n1+n2 == 0:
#                 coefs[feat] = (0, 0)
#             else:
#                 val = (val1*n1 + val2*n2)*1.0/(n1+n2)
#                 n = n1 + n2
#                 coefs[feat] = (val, n)

#     all_coefs = {}
#     for s in outputs[1]:
#         for feat in outputs[1][s]:
#             if feat not in all_coefs:
#                 all_coefs[feat] = (0, 0)

#             val1, n1 = outputs[1][s][feat]
#             val2, n2 = all_coefs[feat]

#             if n1+n2 == 0:
#                 all_coefs[feat] = (0, 0)
#             else:
#                 val = (val1*n1 + val2*n2)*1.0/(n1+n2)
#                 n = n1 + n2
#                 all_coefs[feat] = (val, n)

#     return coefs, all_coefs

# printed_text = ""
# printed_text += "\subsection{Closeness}"+"\n"
# outputs = [
#     outputs_to_dict(shap_feats1),
#     outputs_to_dict(shap_feats3),
#     outputs_to_dict(shap_feats5)
# ]

# printed_text += "\\begin{longtable}[h]{"+"\n"
# printed_text += "    p{\dimexpr 0.40\\linewidth-2\\tabcolsep}|c|c|c|c|c|c|"+"\n"
# printed_text += "}"+"\n"
# printed_text += "    \hline"+"\n"
# printed_text += "    Lexical Features & " + "\n"
# printed_text += "    \\multicolumn{2}{|c|}{Setting 1} & " + "\n"
# printed_text += "    \\multicolumn{2}{|c|}{Setting 2} & " + "\n"
# printed_text += "    \\multicolumn{2}{|c|}{Setting 3} \\\\" + "\n"
# printed_text += "    \\cline{2-7}" + "\n"
# printed_text += "    & " + "\n"
# printed_text += "    Per \\newline token & Total & " + "\n"
# printed_text += "    Per \\newline token & Total & " + "\n"
# printed_text += "    Per \\newline token & Total \\\\" + "\n"

# printed_text += "    \hline"+"\n"
# #     printed_text += "    \endfirsthead"+"\n"
# #     printed_text += ""+"\n"
# printed_text += "    \endhead"+"\n"
# printed_text += ""+"\n"

# # for sec, results in zip(sections, outputs):
# for g in metric_names:
#     if g in ["Conversational Statistics"]:
#         continue

#     printed_text += "    \multicolumn{7}{l}{\\textit{"+g+"}} \\\\"+"\n"
#     printed_text += "    \hline"+"\n"

#     for m in metric_names[g]:
#         s = f"        {metric_names[g][m]} "
#         for out, all_out in outputs:
#             if m not in out:
#                 s += f"& - "
#             else:
#                 val, n = out[m]
#                 all_val, _ = all_out[m]
#                 ref, _ = out["pertoken"]

#                 if (val - ref)*100 > 0.1:
#                     s += "& \cellcolor{gray!25} "+f"{val*100:.2f}"+" & \cellcolor{gray!25} "+f"{all_val*100:.2f}"
#                 else:
#                     s += f"& {val*100:.2f} & {all_val*100:.2f}"

#         s += "\\\\"
#         # print(s)
#         printed_text += s+"\n"


#     printed_text += "        & & & & & &\\\\"+"\n"
#     printed_text += "    \hline"+"\n"
#     printed_text += ""+"\n"

# printed_text += "\label{closeness_wangchanberta_shapley_value}"+"\n"
# printed_text += "\end{longtable}"+"\n"
# printed_text += "\clearpage"+"\n"

In [ ]:
# printed_text += "\subsection{Respect}"+"\n"
# outputs = [
#     outputs_to_dict(shap_feats2),
#     outputs_to_dict(shap_feats4),
#     outputs_to_dict(shap_feats6)
# ]

# printed_text += "\\begin{longtable}[h]{"+"\n"
# printed_text += "    p{\dimexpr 0.40\\linewidth-2\\tabcolsep}|c|c|c|c|c|c|"+"\n"
# printed_text += "}"+"\n"
# printed_text += "    \hline"+"\n"
# printed_text += "    Lexical Features & " + "\n"
# printed_text += "    \\multicolumn{2}{|c|}{Setting 1} & " + "\n"
# printed_text += "    \\multicolumn{2}{|c|}{Setting 2} & " + "\n"
# printed_text += "    \\multicolumn{2}{|c|}{Setting 3} \\\\" + "\n"
# printed_text += "    \\cline{2-7}" + "\n"
# printed_text += "    & " + "\n"
# printed_text += "    Per \\newline token & Total & " + "\n"
# printed_text += "    Per \\newline token & Total & " + "\n"
# printed_text += "    Per \\newline token & Total \\\\" + "\n"

# printed_text += "    \hline"+"\n"
# #     printed_text += "    \endfirsthead"+"\n"
# #     printed_text += ""+"\n"
# printed_text += "    \endhead"+"\n"
# printed_text += ""+"\n"

# # for sec, results in zip(sections, outputs):
# for g in metric_names:
#     if g in ["Conversational Statistics"]:
#         continue

#     printed_text += "    \multicolumn{7}{l}{\\textit{"+g+"}} \\\\"+"\n"
#     printed_text += "    \hline"+"\n"

#     for m in metric_names[g]:
#         s = f"        {metric_names[g][m]} "
#         for out, all_out in outputs:
#             if m not in out:
#                 s += f"& - "
#             else:
#                 val, n = out[m]
#                 all_val, _ = all_out[m]
#                 ref, _ = out["pertoken"]

#                 if (val - ref)*100 > 0.1:
#                     s += "& \cellcolor{gray!25} "+f"{val*100:.2f}"+" & \cellcolor{gray!25} "+f"{all_val*100:.2f}"
#                 else:
#                     s += f"& {val*100:.2f} & {all_val*100:.2f}"

#         s += "\\\\"
#         # print(s)
#         printed_text += s+"\n"


#     printed_text += "        & & & & & &\\\\"+"\n"
#     printed_text += "    \hline"+"\n"
#     printed_text += ""+"\n"
# printed_text += "\label{respect_wangchanberta_shapley_value}"+"\n"
# printed_text += "\end{longtable}"+"\n"

In [ ]:
from collections import defaultdict
from itertools import groupby
from data_loader import preprocess

def get_shap_lexicons(df, raw_shap_values):
    shap_lexicons = {"overall": []}
    label_values = set(df["label"])



    # _tmp = raw_shap_values[:, :]
    shap_data = raw_shap_values.data
    shap_values = raw_shap_values.values

    for _, label in enumerate(label_values):
        feats = []
        for idx, (l, t) in enumerate(zip(df["label"], df["text"])):
            if l!=label:
                continue

            words = word_tokenize(preprocess(t))
            words = [w.strip() for w in words if len(w.strip())>0]

            shap_tokens = [(w.strip(), v) for w,v in zip(shap_data[idx], shap_values[idx]) if len(w.strip())>0]
            shap_tokens = map_token_2_words(words, shap_tokens, debug=False)
            feats.append(shap_tokens)
        shap_lexicons[label] = feats
        shap_lexicons["overall"] += feats

    return shap_lexicons


def is_target_lexicons(token):
  token = token.lower()
  if token=="rep":
      return True

  if token in lexicons_keys:
      for l in lexicons_keys[token]:
          if not token.startswith(l):
              continue

          return True
  return False

def get_shap_by_lexicons(shap_path, fn_abs=False):
    shap_values = load_shap_values(shap_path)

    shap_lexicons = get_shap_lexicons(shap_values["inputs"], shap_values["values"])
    lexicons = {}
    for label in shap_lexicons:

      shapley_values = defaultdict(list)
      for words, shap in shap_lexicons[label]:
        for w, s in zip(words, shap):
          shapley_values[w].append(s)


      feature_names = []
      avg_shape_values = []
      for w in shapley_values:
        if not is_target_lexicons(w):
          continue

        feature_names.append(w)

        if fn_abs:
            s = np.mean(np.abs(shapley_values[w]))
        else:
            s = np.mean((shapley_values[w]))

        avg_shape_values.append(s)

      sorted_shap_values = sorted(zip(avg_shape_values, feature_names), key=lambda pair: -pair[0])
      lexicons[label] = {x.lower():v for v, x in sorted_shap_values}

    return lexicons




In [ ]:
lexicons["rep"] = ["misspelling"]

def get_lexicons_by_feat(shap_lexicons):
  shap_lexicons_by_feat = {}
  for label in shap_lexicons:
    feats = ["pronoun", "particles", "misspelling"]
    shap_lexicons_by_feat[label] = {}
    for f in feats:
      shap_lexicons_by_feat[label][f] = {}
      for w in shap_lexicons[label]:
        tags = lexicons[w]

        if f in lexicons[w]:
          shap_lexicons_by_feat[label][f][w] = shap_lexicons[label][w]
  return shap_lexicons_by_feat

In [ ]:
fn_abs = True
suffix = "abs" if fn_abs else ''

shap_lexicons = get_shap_by_lexicons(f"./ShapleyValuesV3/task1_clse_regressor.pkl", fn_abs)
shap_lexicons_by_feat = get_lexicons_by_feat(shap_lexicons)

dump_jsonl(f"Lexicons/v3/lexicon_task1_clse{suffix}.jsonl", [shap_lexicons, shap_lexicons_by_feat])

Wrote 2 records to Lexicons/v3/lexicon_task1_clseabs.jsonl


In [ ]:
shap_lexicons = get_shap_by_lexicons(f"./ShapleyValuesV3/task2_clse_regressor.pkl", fn_abs)
shap_lexicons_by_feat = get_lexicons_by_feat(shap_lexicons)
dump_jsonl(f"Lexicons/v3/lexicon_task2_clse{suffix}.jsonl", [shap_lexicons, shap_lexicons_by_feat])

Wrote 2 records to Lexicons/v3/lexicon_task2_clseabs.jsonl


In [ ]:
shap_lexicons = get_shap_by_lexicons(f"./ShapleyValuesV3/task3_clse_regressor.pkl", fn_abs)
shap_lexicons_by_feat = get_lexicons_by_feat(shap_lexicons)
dump_jsonl(f"Lexicons/v3/lexicon_task3_clse{suffix}.jsonl", [shap_lexicons, shap_lexicons_by_feat])

Wrote 2 records to Lexicons/v3/lexicon_task3_clseabs.jsonl


In [ ]:
shap_lexicons = get_shap_by_lexicons(f"./ShapleyValuesV3/task1_auth_regressor.pkl", fn_abs)
shap_lexicons_by_feat = get_lexicons_by_feat(shap_lexicons)
dump_jsonl(f"Lexicons/v3/lexicon_task1_auth{suffix}.jsonl", [shap_lexicons, shap_lexicons_by_feat])

Wrote 2 records to Lexicons/v3/lexicon_task1_authabs.jsonl


In [ ]:
shap_lexicons = get_shap_by_lexicons(f"./ShapleyValuesV3/task2_auth_regressor.pkl", fn_abs)
shap_lexicons_by_feat = get_lexicons_by_feat(shap_lexicons)
dump_jsonl(f"Lexicons/v3/lexicon_task2_auth{suffix}.jsonl", [shap_lexicons, shap_lexicons_by_feat])

Wrote 2 records to Lexicons/v3/lexicon_task2_authabs.jsonl


In [ ]:
shap_lexicons = get_shap_by_lexicons(f"./ShapleyValuesV3/task3_auth_regressor.pkl", fn_abs)
shap_lexicons_by_feat = get_lexicons_by_feat(shap_lexicons)
dump_jsonl(f"Lexicons/v3/lexicon_task3_auth{suffix}.jsonl", [shap_lexicons, shap_lexicons_by_feat])

Wrote 2 records to Lexicons/v3/lexicon_task3_authabs.jsonl


## Print outputs

In [ ]:
from pythainlp.util import countthai
import re
# import emoji

def get_lexicon_feats(token, ref_text):
    feats = ["all"]

    if token=="rep":
        feats.append("nrepeat")

    if token in lexicons_keys:
        for l in lexicons_keys[token]:
            if not ref_text.startswith(l):
                continue

            feats.extend(lexicons[l])

    if token in thaidict_royal:
        feats.append("ndict")

    if len(token) > 7:
        feats.append("nlongword")

    if countthai(token) < 50:
        nt = re.sub(r'\W+', '', token)
        if token not in ["usr", "sys", "rep"] and len(nt) > 0 and not nt.isnumeric():
            feats.append("nnotthai")
    else:
        feats.append("nthai")

    if "particles" in feats and "particles_SARP" not in feats:
        feats.append("particles_notSARP")

    # if emoji.emoji_count(token) > 0:
    #     feats.append("nemoji")

    return feats


def get_shap_feats(shap_lexicons):
    output = {}
    for label in shap_lexicons:
        all_shap_feats = []
        for tokens, values in shap_lexicons[label]:
            # shap_feats is per conversation
            shap_feats = defaultdict(list)
            for tidx, (t, v) in enumerate(zip(tokens, values)):
                feats = get_lexicon_feats(t, "".join(tokens[tidx:]))

                for f in feats:
                    shap_feats[f].append(v)

#             shap_feats["pertoken"] = sum(values)/len(values)
            shap_feats["pertoken"] = values
            all_shap_feats.append(shap_feats)

        mean_shap_feats = {}
        for g in metric_names:
            for m in metric_names[g]:
                values_each_token = []
                values_each_conv = []

                for feats in all_shap_feats:
                    if m in feats:
                        values_each_token += feats[m]

                        absum = np.sum(np.abs(np.array(feats[m])))
                        values_each_conv += [absum]
                        #values.append(feats[m])

                if len(values_each_token)==0:
                    mean_shap_feats[m] = (0, 0, 0, 0)
                    continue

                values = np.array(values_each_token)
                pertoken = np.mean(np.abs(np.array(values)))


                perconv = np.mean(np.array(values_each_conv))

                mean_shap_feats[m] = (pertoken, len(values_each_token), perconv, len(values_each_conv))
        output[label] = mean_shap_feats

    return output

def run_lexicons(shap_path):
    shap_values = load_shap_values(shap_path)

    shap_lexicons = get_shap_lexicons(shap_values["inputs"], shap_values["values"])
    shap_feats = get_shap_feats(shap_lexicons)

    return shap_feats



In [ ]:
ls ShapleyValuesV3/

task1_auth_regressor.pkl  task2_auth_regressor.pkl  task3_auth_regressor.pkl
task1_clse_regressor.pkl  task2_clse_regressor.pkl  task3_clse_regressor.pkl


In [ ]:
shap_feats1 = run_lexicons(f"./ShapleyValuesV3/task1_clse_regressor.pkl")
shap_feats2 = run_lexicons(f"./ShapleyValuesV3/task1_auth_regressor.pkl")
shap_feats3 = run_lexicons(f"./ShapleyValuesV3/task2_clse_regressor.pkl")
shap_feats4 = run_lexicons(f"./ShapleyValuesV3/task2_auth_regressor.pkl")
shap_feats5 = run_lexicons(f"./ShapleyValuesV3/task3_clse_regressor.pkl")
shap_feats6 = run_lexicons(f"./ShapleyValuesV3/task3_auth_regressor.pkl")

In [ ]:
def outputs_to_dict(outputs):
    coefs = {}
    all_coefs = {}

    for s in outputs:
        for feat in outputs[s]:
            if feat not in coefs:
                coefs[feat] = (0, 0)

            if feat not in all_coefs:
                all_coefs[feat] = (0, 0)

            # pertoken coefs
            val1, n1, _, _ = outputs[s][feat]
            val2, n2 = coefs[feat]

            if n1+n2 == 0:
                coefs[feat] = (0, 0)
            else:
                val = (val1*n1 + val2*n2)*1.0/(n1+n2)
                n = n1 + n2
                coefs[feat] = (val, n)

            # perconv coefs
            _, _, val1, n1 = outputs[s][feat]
            val2, n2 = all_coefs[feat]

            if n1+n2 == 0:
                all_coefs[feat] = (0, 0)
            else:
                val = (val1*n1 + val2*n2)*1.0/(n1+n2)
                n = n1 + n2
                all_coefs[feat] = (val, n)

    return coefs, all_coefs

printed_text = ""
printed_text += "\subsection{Closeness}"+"\n"
outputs = [
    outputs_to_dict(shap_feats1),
    outputs_to_dict(shap_feats3),
    outputs_to_dict(shap_feats5)
]

printed_text += "\\begin{longtable}[h]{"+"\n"
printed_text += "    p{\dimexpr 0.40\\linewidth-2\\tabcolsep}|c|c|c|c|c|c|"+"\n"
printed_text += "}"+"\n"
printed_text += "    \hline"+"\n"
printed_text += "    Lexical Features & " + "\n"
printed_text += "    \\multicolumn{2}{|c|}{Setting 1} & " + "\n"
printed_text += "    \\multicolumn{2}{|c|}{Setting 2} & " + "\n"
printed_text += "    \\multicolumn{2}{|c|}{Setting 3} \\\\" + "\n"
printed_text += "    \\cline{2-7}" + "\n"
printed_text += "    & " + "\n"
printed_text += "    Per \\newline token & Total & " + "\n"
printed_text += "    Per \\newline token & Total & " + "\n"
printed_text += "    Per \\newline token & Total \\\\" + "\n"

printed_text += "    \hline"+"\n"
#     printed_text += "    \endfirsthead"+"\n"
#     printed_text += ""+"\n"
printed_text += "    \endhead"+"\n"
printed_text += ""+"\n"

# for sec, results in zip(sections, outputs):
for g in metric_names:
    if g in ["Conversational Statistics"]:
        continue

    printed_text += "    \multicolumn{7}{l}{\\textit{"+g+"}} \\\\"+"\n"
    printed_text += "    \hline"+"\n"

    for m in metric_names[g]:
        s = f"        {metric_names[g][m]} "
        for out, all_out in outputs:
            if m not in out:
                s += f"& - "
            else:
                val, n = out[m]
                all_val, _ = all_out[m]
                ref, _ = out["pertoken"]

                if (val - ref) > 0.1*ref:
                    s += "& \cellcolor{gray!25} "+f"{val*100:.2f}"+" & \cellcolor{gray!25} "+f"{all_val*100:.2f}"
                else:
                    s += f"& {val*100:.2f} & {all_val*100:.2f}"

        s += "\\\\"
        # print(s)
        printed_text += s+"\n"


    printed_text += "        & & & & & &\\\\"+"\n"
    printed_text += "    \hline"+"\n"
    printed_text += ""+"\n"

printed_text += "\label{closeness_wangchanberta_shapley_value}"+"\n"
printed_text += "\end{longtable}"+"\n"
printed_text += "\clearpage"+"\n"

In [ ]:
printed_text += "\subsection{Respect}"+"\n"
outputs = [
    outputs_to_dict(shap_feats2),
    outputs_to_dict(shap_feats4),
    outputs_to_dict(shap_feats6)
]

printed_text += "\\begin{longtable}[h]{"+"\n"
printed_text += "    p{\dimexpr 0.40\\linewidth-2\\tabcolsep}|c|c|c|c|c|c|"+"\n"
printed_text += "}"+"\n"
printed_text += "    \hline"+"\n"
printed_text += "    Lexical Features & " + "\n"
printed_text += "    \\multicolumn{2}{|c|}{Setting 1} & " + "\n"
printed_text += "    \\multicolumn{2}{|c|}{Setting 2} & " + "\n"
printed_text += "    \\multicolumn{2}{|c|}{Setting 3} \\\\" + "\n"
printed_text += "    \\cline{2-7}" + "\n"
printed_text += "    & " + "\n"
printed_text += "    Per \\newline token & Total & " + "\n"
printed_text += "    Per \\newline token & Total & " + "\n"
printed_text += "    Per \\newline token & Total \\\\" + "\n"

printed_text += "    \hline"+"\n"
#     printed_text += "    \endfirsthead"+"\n"
#     printed_text += ""+"\n"
printed_text += "    \endhead"+"\n"
printed_text += ""+"\n"

# for sec, results in zip(sections, outputs):
for g in metric_names:
    if g in ["Conversational Statistics"]:
        continue

    printed_text += "    \multicolumn{7}{l}{\\textit{"+g+"}} \\\\"+"\n"
    printed_text += "    \hline"+"\n"

    for m in metric_names[g]:
        s = f"        {metric_names[g][m]} "
        for out, all_out in outputs:
            if m not in out:
                s += f"& - "
            else:
                val, n = out[m]
                all_val, _ = all_out[m]
                ref, _ = out["pertoken"]

                if (val - ref) > 0.1*ref:
                    s += "& \cellcolor{gray!25} "+f"{val*100:.2f}"+" & \cellcolor{gray!25} "+f"{all_val*100:.2f}"
                else:
                    s += f"& {val*100:.2f} & {all_val*100:.2f}"

        s += "\\\\"
        # print(s)
        printed_text += s+"\n"


    printed_text += "        & & & & & &\\\\"+"\n"
    printed_text += "    \hline"+"\n"
    printed_text += ""+"\n"
printed_text += "\label{respect_wangchanberta_shapley_value}"+"\n"
printed_text += "\end{longtable}"+"\n"

In [ ]:
print(printed_text)

\subsection{Closeness}
\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\tabcolsep}|c|c|c|c|c|c|
}
    \hline
    Lexical Features & 
    \multicolumn{2}{|c|}{Setting 1} & 
    \multicolumn{2}{|c|}{Setting 2} & 
    \multicolumn{2}{|c|}{Setting 3} \\
    \cline{2-7}
    & 
    Per \newline token & Total & 
    Per \newline token & Total & 
    Per \newline token & Total \\
    \hline
    \endhead

    \multicolumn{7}{l}{\textit{Reference}} \\
    \hline
        Average per token & 0.63 & 43.13& 2.05 & 55.88& 0.51 & 34.77\\
        & & & & & &\\
    \hline

    \multicolumn{7}{l}{\textit{Pronoun}} \\
    \hline
        All pronoun & \cellcolor{gray!25} 0.77 & \cellcolor{gray!25} 2.32& 2.18 & 4.53& \cellcolor{gray!25} 1.04 & \cellcolor{gray!25} 3.05\\
        >> 1st person pronoun & 0.66 & 1.39& \cellcolor{gray!25} 2.45 & \cellcolor{gray!25} 3.66& \cellcolor{gray!25} 0.65 & \cellcolor{gray!25} 1.33\\
        >> 2nd person pronoun & \cellcolor{gray!25} 0.91 & \cellcolor{gray!25} 2.01&